In [77]:
# import dependancies
import pandas as pd
import random
from datetime import datetime, timedelta
from yahoo_fin.stock_info import *
import yfinance as yf
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from splinter import Browser
from pprint import pprint
import finnhub
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

# change to your api keys 

#alphavantage_api_key = 'AJP4EKR4LE52NNK4'
alphavantage_api_key = 'CORYDT1RMNVL3DVT'
polygon_api_key = 'J9UUfkyTylf3D_s76b5pcMCnLSaKTNpD'
finnhub_client = finnhub.Client(api_key="cld35g9r01qn959jif1gcld35g9r01qn959jif20")


### Screening US Mega & Large Cap Stocks

In [78]:
file_path = 'nasdaq_screener_1700335987448.csv'

# Read the CSV file into a Pandas DataFrame
stock_list_df = pd.read_csv(file_path)

# Display the DataFrame
stock_list_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$113.15,-1.04,-0.91%,3.310625e+10,United States,1999.0,2700770,Industrials,Electrical Products
1,AAPL,Apple Inc. Common Stock,$189.69,-0.02,-0.01%,2.950200e+12,United States,1980.0,50922657,Technology,Computer Manufacturing
2,ABBV,AbbVie Inc. Common Stock,$138.30,0.02,0.01%,2.441740e+11,United States,2012.0,4332297,Health Care,Biotechnology: Pharmaceutical Preparations
3,ABNB,Airbnb Inc. Class A Common Stock,$127.15,0.87,0.69%,8.244890e+10,United States,2020.0,3795704,Consumer Discretionary,Hotels/Resorts
4,ABT,Abbott Laboratories Common Stock,$99.55,-0.71,-0.71%,1.728250e+11,United States,NaN,6069070,Health Care,Biotechnology: Pharmaceutical Preparations


In [79]:
# create an array of tickers

ticker_list = stock_list_df['Symbol'].values

### Generating functions to retrieve stock information 

In [51]:
# define function to generate random date, and ensure that the date is a weekday, maybe start in 2014 or 2015

def generate_random_date(start_date, end_date):
    check_weekday = False
    while not check_weekday:
        # Convert the date strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date_dt = datetime.datetime.now()  # Using today's date as the end date

        # Calculate the range of days between the start and end date
        delta_days = (end_date_dt - start_date_dt).days

        # Generate a random number of days within the range
        random_days = random.randint(0, delta_days)

        # Calculate the random date
        random_date = start_date_dt + timedelta(days=random_days)

        # Check if the generated date is a weekday
        if random_date.weekday() < 5:
            check_weekday = True
            return random_date
        else:
            check_weekday = False



In [52]:
# define a function to choose a random stock from the list

def choose_random_stock(ticker_list):    
    
    # Randomly choose a stock from the list
    random_stock = random.choice(ticker_list)
    
    return random_stock.lower()
    

In [53]:
# define a function to check if data exists for stock on the date 

def check_stock_date(ticker, check_date):
    check_financials = False
    # Check if the input date is a Friday
    if check_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        adj_end_date = check_date + timedelta(days=3)
    else:
        # Set end date to the next day
        adj_end_date = check_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    
    
    try:
        get_data(ticker, end_date=adj_end_date)
        get_stock_financials(ticker, check_date)
        

        check_financials = True
    except:
        check_financials = False
    
    if not check_financials:
        return False
    else:
        return True
    

In [54]:
# define a function to return stock industry

def return_stock_industry(symbol):
    symbol = symbol.upper()
    # Check if the symbol is in the DataFrame
    if symbol in stock_list_df['Symbol'].values:
        # Retrieve the sector for the given symbol
        sector = stock_list_df.loc[stock_list_df['Symbol'] == symbol, 'Sector'].iloc[0]
        return sector
    else:
        return f"Symbol '{symbol}' not found in the stock list."

In [55]:

# define a function to get the adjusted closing price of a stock on a particular day

def get_stock_price(ticker, stock_date):
    # Check if the input date is a Friday
    if stock_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        price_adj_end_date = stock_date + timedelta(days=3)
    else:
        # Set end date to the next day
        price_adj_end_date = stock_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=price_adj_end_date)
    # extracting the price 
    adj_close_price = stock_data['adjclose'].values[-1]
    
    return adj_close_price

In [56]:
# define a function to get stock trading volume

def get_stock_volume(ticker, vol_date):
    # Check if the input date is a Friday
    if vol_date.weekday() == 4:  # Friday
        # Set end date to the next Monday
        vol_adj_end_date = vol_date + timedelta(days=3)
    else:
        # Set end date to the next day
        vol_adj_end_date = vol_date + timedelta(days=1)
    
    # using yahoo finance function to get stock data dataframe
    stock_data = get_data(ticker, end_date=vol_adj_end_date)
    
    #extracting the volume
    trading_vol = stock_data['volume'].values[-1]
    
    return trading_vol

In [57]:
# function to get 4 previous weekdays from given_date

def get_previous_weekdays(weekday_date):
    previous_weekdays = []
    previous_date = weekday_date - timedelta(days=1)
    while len(previous_weekdays) < 4:
        if previous_date.weekday() < 5:
            previous_weekdays.append(previous_date)
        previous_date -= timedelta(days=1)

    return previous_weekdays


In [97]:
# function to get company's financial ratios by scraping stockanalysis.com

def get_stock_financials(ticker, fin_date):
    
    financial_ratios = {}

    # setting brwoser
    browser = Browser('chrome')
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/ratios/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
        # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        # Extract date columns excluding the first column
        date_columns = df.columns[2:-1]
        
        # Convert date columns to datetime objects
        date_columns = pd.to_datetime(date_columns)
        
        # Find the closest date column to the given date that is earlier
        earlier_dates = [date for date in date_columns if date < fin_date]
        closest_date_column = max(earlier_dates)

        # Access the closest column
        closest_column = closest_date_column.strftime('%Y-%m-%d')
        
    except:
        print("Table not found on the page.")
    
    

    #extracting financial ratios
    try: 
        financial_ratios['pe_ratio']=float(df.loc[df['Quarter Ended'] == 'PE Ratio', closest_column].values[0])
        financial_ratios['debt_to_equity']=float(df.loc[df['Quarter Ended'] == 'Debt / Equity Ratio', closest_column].values[0])
        financial_ratios['quick_ratio']=float(df.loc[df['Quarter Ended'] == 'Quick Ratio', closest_column].values[0])
        total_shareholder_return_str = df.loc[df['Quarter Ended'] == 'Total Shareholder Return', closest_column].values[0]
        financial_ratios['total_shareholder_return'] = float(total_shareholder_return_str.strip('%'))
    
    except:
        pass 
    
    # URL of the website
    url = f"https://stockanalysis.com/stocks/{ticker}/financials/?p=quarterly"

    # Use Selenium to open the webpage and let JavaScript execute
    driver = webdriver.Chrome()  # Make sure you have the ChromeDriver executable in your PATH
    driver.get(url)

    # Allow some time for JavaScript to execute and load the content
    time.sleep(5)  # Adjust the sleep time based on your needs

    # Get the page source after JavaScript execution
    page_source = driver.page_source

    # Close the Selenium WebDriver
    driver.quit()

    # Parse the HTML content of the page
    soup = BeautifulSoup(page_source, "html.parser")

    # Find the table by class name
    table = soup.find("table", class_="w-full")

    # Check if the table is found
    try:
    # Use pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0]
        
    except:
        print("Table not found on the page.")
    
    try:
        profit_margin_str= df.loc[df['Quarter Ended'] == 'Profit Margin', closest_column].values[0]
        financial_ratios['profit_margin'] = float(profit_margin_str.strip('%'))
        cash_margin_str= df.loc[df['Quarter Ended'] == 'Free Cash Flow Margin', closest_column].values[0]
        financial_ratios['free_cash_margin'] = float(cash_margin_str.strip('%'))
    except:
        pass

    return financial_ratios
    
    

In [59]:
# function to calculate stock's volatility

def calculate_volatility(ticker, sd_date):
    df=get_data(ticker, end_date=sd_date)
    # Calculate daily returns
    df['daily_return'] = df['adjclose'].pct_change()

    # Calculate volatility as the standard deviation of daily returns
    volatility = df['daily_return'].std()

    return volatility

In [13]:
#------------------------NEED TO FIND A NEW SOURCE---------------------------------

def get_news_sentiment(ticker, news_date):
    
    # Extract year, month, and day
    news_end_date=news_date
    end_year = news_end_date.year
    end_month = str(news_end_date.month).zfill(2)
    end_day = str(news_end_date.day).zfill(2)
    
    news_start_date = news_end_date - timedelta(weeks=4)
    start_year = news_start_date.year
    start_month = str(news_start_date.month).zfill(2)
    start_day = str(news_start_date.day).zfill(2)
   

In [60]:
# extracting daily federal interest rate information from alphavantage

# using alphavantage API
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={alphavantage_api_key}'
r = requests.get(url)
data = r.json()

daily_interest_rates = data

# converting json to dataframe
daily_interest_rate_df = pd.DataFrame(daily_interest_rates['data'])

# Export DataFrame to CSV so that we can refer to it without making an api call
daily_interest_rate_df.to_csv('daily_interest_rates.csv', index=False)


In [61]:
# extracting monthly CPI information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=CPI&interval=monthly&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_cpi = data

# converting json to dataframe
monthly_cpi_df = pd.DataFrame(monthly_cpi['data'])
monthly_cpi_df['date'] = pd.to_datetime(monthly_cpi_df['date'])


# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_cpi_df.to_csv('monthly_cpi.csv', index=False)

In [62]:
# extracting monthly unemployment rate information from alphavantage

# using alphavantage api
url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey=demo'
r = requests.get(url)
data = r.json()

monthly_unemployment_rate = data

# converting json to dataframe
monthly_unemployment_rate_df = pd.DataFrame(monthly_unemployment_rate['data'])
monthly_unemployment_rate_df['date'] = pd.to_datetime(monthly_unemployment_rate_df['date'])

# Export DataFrame to CSV so that we can refer to it without making an api call
monthly_unemployment_rate_df.to_csv('monthly_unemployment_rate.csv', index=False)

In [63]:
# define a function to extract economic indicators for the given date

def get_economic_indicators(econ_date):
    
    economic_indicators_dict={}
    
    # convert datetime object to matching string 
    formatted_date = econ_date.strftime('%Y-%m-%d')
    
    # retrieve CPI value
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['cpi'] = float(monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])    
    
    # retrieve federal interest rate value
    economic_indicators_dict['interest_rate'] = float(daily_interest_rate_df.loc[daily_interest_rate_df['date'] == formatted_date, 'value'].iloc[0])

    # retrieve unemployement rate
    # Locate the value for the given date, matching only month and year
    economic_indicators_dict['unemployment_rate'] = float(monthly_unemployment_rate_df.loc[monthly_unemployment_rate_df['date'].dt.to_period('M') == pd.to_datetime(econ_date).to_period('M'), 'value'].iloc[0])
    
    return economic_indicators_dict

In [64]:
# function to retrieve technical indicators

def get_technical_indicators(ticker, tech_date):
    
    technical_indicators_dict={}
    
    # changing date into UNIX format
    end_timestamp = int(tech_date.timestamp())
    
    # choosing a timestamp of one week earlier
    start_tech_date = tech_date - timedelta(weeks=1)
    
    # changing the date to UNIX format
    start_timestamp = int(start_tech_date.timestamp())
    
    # retrieving SMA 
    technical_indicators_dict['sma']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='sma', indicator_fields={"timeperiod": 5})['sma'][-1]
    
    #retrieving RSI
    technical_indicators_dict['rsi']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='rsi', indicator_fields={"timeperiod": 5})['rsi'][-1]

    # retrieving EMA 
    technical_indicators_dict['ema']= finnhub_client.technical_indicator(symbol=ticker, resolution='D', _from=start_timestamp, to=end_timestamp, indicator='ema', indicator_fields={"timeperiod": 5})['ema'][-1]
    
    return technical_indicators_dict

In [65]:
# function to create label value for data

def create_label(ticker, y_date):
    print(y_date)
    initial_price = get_stock_price(ticker, y_date)
    print(initial_price)
    
    # find the next weekday
    
    # check if date landed on friday and calculate the next weekday date
    if y_date.weekday() == 4:
        next_weekday_date = y_date + timedelta(days=3)
    else:
        next_weekday_date = y_date + timedelta(days=1)

    print(next_weekday_date)
    
    next_price = get_stock_price(ticker, next_weekday_date)
    
    print(next_price)
    
    if next_price >= initial_price:
        return 1 # 1 is class increase, in order to simplify the classification we put stayed the same in same category as increase
    else:
        return 0 # 0 is class decrease

### Running a loop to generate multiple rows of data for training and testing

In [21]:
page=10

In [106]:
# list to store dictionaries
list_dict = []
page += 1
# runs loop until list_dic [change no. 4 as desired]
while len(list_dict) < 500:

    # data_dict
    data_dict = {}
    
    # checking to see if generated ticker-date combo has available data
    checker = False
    
    # runs this loop until checker is true
    while not checker:

        ticker = choose_random_stock(ticker_list)
        input_date = generate_random_date('2015-01-01', '2023-10-31')

        checker = check_stock_date(ticker, input_date)

    # using function to return stock industry
    data_dict['industry']=return_stock_industry(ticker)

    # retrieve weekly returns
    date = input_date
    for i in range(1, 5):
        data_dict[f'wr{i}']=(get_stock_price(ticker, date) - get_stock_price(ticker, date - timedelta(weeks = 1)))/get_stock_price(ticker, date - timedelta(weeks = 1))
        date = date - timedelta(weeks=1)

    # retrieve last 5 working day volumes
    data_dict['vol1'] = get_stock_volume(ticker, date)
    for i, weekday in enumerate(get_previous_weekdays(input_date)):
        data_dict[f'vol{i+1}'] = get_stock_volume(ticker, weekday)

    # retrieve stock financials    
    for key, value in get_stock_financials(ticker, input_date).items():
        data_dict[key] = value
    
    # retrieve stock volatility
    data_dict['volatility'] = calculate_volatility(ticker, input_date)

    # retrieve economic indicators
    for key, value in get_economic_indicators(input_date).items():
        data_dict[key] = value

    # retrieve stocktechnical indicators    
    for key, value in get_technical_indicators(ticker, input_date).items():
        data_dict[key] = value

    # generate a label for row    
    data_dict['label'] = create_label(ticker, input_date)
        
    # append dictionary to list
    list_dict.append(data_dict)
    print(len(list_dict))

C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2017-08-16 00:00:00
94.00019836425781
2017-08-17 00:00:00
92.4930191040039
1


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2015-09-23 00:00:00
57.024356842041016
2015-09-24 00:00:00
56.16374206542969
2


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

...

2015-11-17 00:00:00
15.041518211364746
2015-11-18 00:00:00
15.366368293762207
82


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

2015-09-24 00:00:00
20.251514434814453
2015-09-25 00:00:00
20.46082878112793
83


C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:34: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
C:\Users\gayat\AppData\Local\Temp\ipykernel_23816\3272197425.py:89: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read fr

IndexError: single positional indexer is out-of-bounds

In [84]:
data_dict

{'industry': 'Consumer Discretionary',
 'wr1': -0.011338714161929133,
 'wr2': 0.040786776473789525,
 'wr3': -0.04058799531456262,
 'wr4': -0.049109129017416805,
 'vol1': 10195800,
 'vol2': 10247900,
 'vol3': 11862300,
 'vol4': 12886300}

In [86]:
print(ticker, input_date)

dis 2021-12-14 00:00:00


In [102]:
pd.to_datetime(input_date).to_period('M')

Period('2023-11', 'M')

In [101]:
monthly_cpi_df.loc[monthly_cpi_df['date'].dt.to_period('M') == pd.to_datetime(input_date).to_period('M'), 'value'].iloc[0]  

IndexError: single positional indexer is out-of-bounds

In [104]:
list_dict

[{'industry': 'Consumer Discretionary',
  'wr1': 0.005184658868499337,
  'wr2': -0.017093115708889724,
  'wr3': 0.056533940877884416,
  'wr4': -0.05501372708795849,
  'vol1': 3192200,
  'vol2': 3973700,
  'vol3': 3718400,
  'vol4': 3711300,
  'pe_ratio': 28.36,
  'debt_to_equity': 0.69,
  'quick_ratio': 0.21,
  'total_shareholder_return': 3.26,
  'profit_margin': 13.44,
  'free_cash_margin': 13.55,
  'volatility': 0.02113812995791535,
  'cpi': 296.276,
  'interest_rate': 1.58,
  'unemployment_rate': 3.5,
  'sma': 93.42333333333335,
  'rsi': 56.14678899082585,
  'ema': 93.24666666666667,
  'label': 0},
 {'industry': 'Health Care',
  'wr1': 0.055094028442556925,
  'wr2': -0.0381637825361354,
  'wr3': -0.04009939986851761,
  'wr4': -0.028963134024753263,
  'vol1': 7395317,
  'vol2': 8888193,
  'vol3': 13994154,
  'vol4': 12779102,
  'pe_ratio': 25.56,
  'debt_to_equity': 0.94,
  'quick_ratio': 0.88,
  'total_shareholder_return': 6.45,
  'profit_margin': 26.95,
  'free_cash_margin': 6.85,


### Export data into csv

In [107]:
# remember to export data as csv
data_df = pd.DataFrame(list_dict)
data_df.to_csv(f'data{page}.csv', index=False)


In [108]:
data_df

,industry,wr1,wr2,wr3,wr4,vol1,vol2,vol3,vol4,pe_ratio,...,profit_margin,free_cash_margin,volatility,cpi,interest_rate,unemployment_rate,sma,rsi,ema,label
0,Industrials,0.028500,0.010336,-0.009173,-0.006195,1985800,2424900,2077900,2058200,26.87,...,8.69,8.25,0.014264,245.519,1.16,4.4,114.210000,76.349206,114.085000,0
1,Consumer Discretionary,0.024190,0.021134,0.015025,0.012463,1162200,1016600,1702200,840000,25.07,...,11.57,66.31,0.020739,237.945,0.14,5.0,58.263333,82.222222,58.235833,0
2,Real Estate,0.008714,0.037955,-0.021339,0.007361,1779800,1222500,983000,1696800,129.61,...,6.18,-9.44,0.020941,257.346,1.83,3.6,128.790000,66.889341,129.005417,1
3,Industrials,0.013126,-0.010502,0.048885,-0.046429,307700,504600,921500,695900,5.94,...,22.33,23.30,0.025233,278.802,0.08,3.9,159.776667,65.581395,160.000000,1
4,Consumer Discretionary,-0.034466,-0.034474,0.014081,0.008323,777000,479900,1041100,382500,37.84,...,3.51,-9.33,0.024751,240.229,0.37,4.8,38.670000,41.688244,38.822500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,Technology,-0.029184,-0.029167,-0.004149,-0.070216,1172900,806200,747400,579200,-9.85,...,-40.02,-33.41,0.032467,242.839,0.55,4.7,11.363333,9.523810,11.393333,0
79,Technology,-0.028559,0.019100,0.021207,0.041432,1113900,1753700,1598300,659800,-24.58,...,-50.77,15.77,0.028907,244.733,0.91,4.4,66.773333,21.241288,66.258333,1
80,Technology,-0.042352,-0.113213,0.019138,0.048519,2413100,3346900,4357000,3315300,12.42,...,18.04,24.97,0.037238,237.945,0.13,5.0,62.570000,35.148692,62.787500,1
81,Technology,-0.017363,-0.128363,0.069544,-0.023991,23731800,21071200,14296800,16199800,6.20,...,13.08,-20.42,0.040100,237.336,0.13,5.1,15.056667,55.275591,15.170625,1


In [21]:
page = 1

In [95]:
print(ticker, input_date)

psa 2018-03-14 00:00:00
